In [1]:
import pandas as pd
import numpy as np
import string
import pickle
import io
import torch
import json
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
metadata = pd.read_csv("Movies_dataset.csv")

with open('encodings_description.pickle','rb') as file:
    encoded_data = pickle.load(file)

In [3]:
metadata.head()

,Unnamed: 0,movieId,imdbId,tmdbId,title,overview,budget,genres,production_companies,production_countries,release_date,runtime,spoken_languages,keywords,cast,crew,final_poster_url
0,0,1,114709,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",NF
1,1,2,113497,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",https://image.tmdb.org/t/p/w500/vzmL6fP7aPKNKP...
2,2,3,113228,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",NF
3,3,4,114885,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",NF
4,4,5,113041,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...,0.0,"[{'id': 35, 'name': 'Comedy'}]","[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",NF


In [4]:
mIDs = metadata.loc[:,'movieId'].values
print(mIDs.shape)

(44434,)


In [5]:
np.unique(mIDs).shape

(44434,)

In [6]:
cos_sim_matrix = cosine_similarity(encoded_data)

In [7]:
cos_sim_matrix.shape

(44434, 44434)

In [8]:
len(encoded_data)

44434

In [23]:
top_10_neighbors_dict = {}

# Loop through each item
for i in range(len(encoded_data)):

    # Sort the cosine similarity values for the current item
    sorted_indices = np.argsort(-cos_sim_matrix[i])  # Sort in descending order

    sorted_indices = np.delete(sorted_indices, np.where(sorted_indices == i))

    # Get the indices of the top 10 neighbors (excluding the item itself)
    top_10_indices = sorted_indices[:10]

    # Get the item IDs of the top 10 neighbors
    top_10_item_ids = [str(mIDs[idx]) for idx in top_10_indices]

    # Store the top 10 neighbors in the dictionary

    top_10_neighbors_dict[str(mIDs[i])] = top_10_item_ids

In [24]:
for k,v in top_10_neighbors_dict.items():
    if int(k) in v:
        print(k,v)

In [26]:
# Save dictionary to a JSON file
with open('top_10_neighbors.json', 'w') as json_file:
    json.dump(top_10_neighbors_dict, json_file)

# Load dictionary from the JSON file
with open('top_10_neighbors.json', 'r') as json_file:
    loaded_data = json.load(json_file)

print(loaded_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
len(loaded_data.keys())

44434